# It looks like that the polarization vectors computed by phonopy includes the phase factor exp(iQd)

In [1]:
# some goodies
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np, histogram.hdf as hh, histogram as H
import os

/home/lj7/miniconda2/envs/dev-mcvine-new/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Phonon data

In [2]:
ls ../data/graphite/

exp_DOS.h5  FORCE_CONSTANTS  POSCAR  SPOSCAR


In [4]:
disp_dir = os.path.abspath('../data/graphite/')

In [5]:
# cat {disp_dir}/POSCAR
# cat {disp_dir}/SPOSCAR

## basis and reciprocal basis

In [6]:
basis = np.array(
    map(float,
        "2.44618916393	0.0	0.0	-1.22309458197	2.11846195843	0.0	0.0	0.0	6.65422904762".split())
    )
basis.shape = 3, 3
basis

array([[ 2.44618916,  0.        ,  0.        ],
       [-1.22309458,  2.11846196,  0.        ],
       [ 0.        ,  0.        ,  6.65422905]])

In [7]:
# atom positions
fractional_positions = np.array([
    [0, 0, .25],
    [0, 0, .75],
    [1./3, 2./3, 0.25],
    [2./3, 1./3, 0.75]
])

positions = np.dot(fractional_positions, basis)
positions.shape

(4, 3)

In [8]:
a1,a2,a3 = basis
v = np.dot(a1, np.cross(a2, a3))
b1 = 2*np.pi * np.cross(a2, a3)/v
b2 = 2*np.pi * np.cross(a3, a1)/v
b3 = 2*np.pi * np.cross(a1, a2)/v
Q_basis = np.array([b1, b2, b3])

In [9]:
inv_Q_basis = np.linalg.inv(Q_basis)

## Compute phonons

In [62]:
!cat {os.path.join(disp_dir, 'POSCAR')}

C
    1.0
       2.44618916393 0.00000000000   0.00000000000           
      -1.22309458197 2.11846195843   0.00000000000
       0.00000000000   0.00000000000 6.65422904762 
  4
Direct
        0.00000000 0.00000000 0.25000000
        0.00000000 0.00000000 0.75000000
        0.33333333 0.66666667 0.25000000
        0.66666667 0.33333333 0.75000000


In [65]:
!head -n 20 {os.path.join(disp_dir, 'FORCE_CONSTANTS')}

 144
   1   1
    64.722023959334848    -0.000000000000012     0.000000000000000
    -0.000000000000014    64.722023959334820    -0.000000000000002
     0.000000000000026     0.000000000000012    14.545125718328633
   1   2
    -4.739575929473816    -0.339247569041192     0.000000000000000
     0.340599895562029     2.733796249187515    -0.000000000000000
    -0.000000000000002     0.000000000000000     0.499816270448648
   1   3
    -0.109879716364381     0.034454683900044     0.000000000000000
    -0.035907177193926     0.289124041307858    -0.000000000000000
    -0.000000000000000     0.000000000000000     0.013945451137851
   1   4
    -0.151779628456694     0.000000000000000    -0.000000000000000
    -0.000000000000141    -0.092080491420462     0.000000000000000
    -0.000000000000000    -0.000000000000000    -0.005529744734835
   1   5
    -0.109879716364218    -0.034454683900044     0.000000000000000
     0.035907177192043     0.289124041307695    -0.000000000000000


In [10]:
from phonopy.interface import vasp
from phonopy.units import VaspToTHz
from phonopy import Phonopy, file_IO

bulk = vasp.read_vasp(os.path.join(disp_dir, 'POSCAR'), ['C'])
phonon = Phonopy(bulk, supercell_matrix=np.diag([6,6,1]), factor=VaspToTHz)
phonon.generate_displacements(distance=0.01)
force_constants = file_IO.parse_FORCE_CONSTANTS(os.path.join(disp_dir, 'FORCE_CONSTANTS'))
phonon.set_force_constants(force_constants)

## Polarization vectors: phase

It looks like the pol vectors from phonopy includes the phase factor. let us check

In [85]:
hkls2 = [[0,0,0], [0,0,1]]
phonon.set_qpoints_phonon(hkls2, is_eigenvectors=True, write_dynamical_matrices=False)
freqs2, pols2 = phonon.get_qpoints_phonon()
print pols2.shape
pols2 = np.transpose(pols2, (0,2,1))
pols2.shape = 2, 12, 4, 3

(2, 12, 12)


In [86]:
print freqs2[0]
assert (freqs2[0] == freqs2[1]).all()

[3.34225831e-02 3.34225831e-02 9.28652901e-02 1.52100901e+00
 1.52100901e+00 2.61040097e+00 2.63887591e+01 2.65558347e+01
 4.76904068e+01 4.76904068e+01 4.78045328e+01 4.78045328e+01]


Energies match apparantly. Next: polarizations

In [87]:
pols2[0][2]

array([[ 3.54763646e-13+0.j, -9.11045243e-13+0.j,  5.00000874e-01+0.j],
       [ 3.56783533e-13+0.j, -9.24304724e-13+0.j,  5.00000874e-01+0.j],
       [ 3.54855214e-13+0.j, -9.10807688e-13+0.j,  4.99999126e-01+0.j],
       [ 3.56935177e-13+0.j, -9.24235284e-13+0.j,  4.99999126e-01+0.j]])

In [88]:
pols2[1][2]

array([[-3.54763646e-13+0.j,  9.11045243e-13+0.j, -5.00000874e-01+0.j],
       [ 3.56783533e-13+0.j, -9.24304724e-13+0.j,  5.00000874e-01+0.j],
       [-3.54855214e-13+0.j,  9.10807688e-13+0.j, -4.99999126e-01+0.j],
       [ 3.56935177e-13+0.j, -9.24235284e-13+0.j,  4.99999126e-01+0.j]])

They don't match. for q=000, it is .5, .5, .5, .5. but for q=001, it is -.5, .5, -.5, .5

Next we adjust the phase factor and see if they match

In [89]:
pols2[1][2] * np.exp(1j*np.pi*2* np.dot(hkls2[1], fractional_positions.T))[:, np.newaxis]

array([[-2.17230082e-29-3.54763646e-13j,  5.57854321e-29+9.11045243e-13j,
        -3.06162235e-17-5.00000874e-01j],
       [-6.55400717e-29-3.56783533e-13j,  1.69792023e-28+9.24304724e-13j,
        -9.18486704e-17-5.00000874e-01j],
       [-2.17286151e-29-3.54855214e-13j,  5.57708860e-29+9.10807688e-13j,
        -3.06161165e-17-4.99999126e-01j],
       [-6.55679283e-29-3.56935177e-13j,  1.69779267e-28+9.24235284e-13j,
        -9.18483495e-17-4.99999126e-01j]])

In [93]:
np.allclose(
    pols2[1][2] * np.exp(1j*np.pi*2* np.dot(hkls2[1], fractional_positions.T))[:, np.newaxis]  * 1j,
    pols2[0][2],
)

True

It matches! Note that an overall extra phase factor is fine (1j)

# More testing

In [45]:
hkls2 = [[0,0,0], [0,1,0]]
phonon.set_qpoints_phonon(hkls2, is_eigenvectors=True, write_dynamical_matrices=False)
freqs2, pols2 = phonon.get_qpoints_phonon()
print pols2.shape
pols2 = np.transpose(pols2, (0,2,1))
pols2.shape = 2, 12, 4, 3

(2, 12, 12)


In [58]:
print freqs2.shape
print freqs2[0]
assert np.allclose(freqs2[0], freqs2[1])

(2, 12)
[3.34225831e-02 3.34225831e-02 9.28652901e-02 1.52100901e+00
 1.52100901e+00 2.61040097e+00 2.63887591e+01 2.65558347e+01
 4.76904068e+01 4.76904068e+01 4.78045328e+01 4.78045328e+01]


Energies match apparantly. Next: polarizations

In [47]:
pols2[0][2]

array([[ 3.54763646e-13+0.j, -9.11045243e-13+0.j,  5.00000874e-01+0.j],
       [ 3.56783533e-13+0.j, -9.24304724e-13+0.j,  5.00000874e-01+0.j],
       [ 3.54855214e-13+0.j, -9.10807688e-13+0.j,  4.99999126e-01+0.j],
       [ 3.56935177e-13+0.j, -9.24235284e-13+0.j,  4.99999126e-01+0.j]])

In [48]:
pols2[1][2]

array([[-2.73855177e-12+0.00000000e+00j,  3.93856247e-13+1.91944702e-12j,
        -4.83545889e-01+1.27217319e-01j],
       [-2.72549634e-12+1.26101100e-15j,  3.89695294e-13+1.91233730e-12j,
        -4.83545889e-01+1.27217319e-01j],
       [ 1.36931210e-12-2.37176272e-12j, -1.85917816e-12-6.19048638e-13j,
         1.31599045e-01-4.82370001e-01j],
       [ 1.36391507e-12+2.35975490e-12j,  1.46135079e-12-1.29360880e-12j,
         3.51945138e-01+3.55153131e-01j]])

They don't match. 

Next we adjust the phase factor and see if they match

In [74]:
np.exp(-1j*2*np.pi*np.dot(hkls2[1], fractional_positions.T))

array([ 1. +0.j       ,  1. +0.j       , -0.5+0.8660254j, -0.5-0.8660254j])

In [81]:
print pols2[1][2] * np.exp(1j*np.pi*2* np.dot(hkls2[1], fractional_positions.T))[:, np.newaxis] * .5/(-4.83545889e-01+1.27217319e-01j),

[[2.64842165e-12+6.96779995e-13j 1.07478295e-13-1.95648527e-12j
  5.00000000e-01-1.04971462e-10j]
 [2.63611674e-12+6.92238743e-13j 1.09693350e-13-1.94855085e-12j
  5.00000000e-01-1.04972456e-10j]
 [2.64853472e-12+6.96786689e-13j 1.07888803e-13-1.95655597e-12j
  4.99998253e-01-1.05769097e-08j]
 [2.63619522e-12+6.92211162e-13j 1.09772219e-13-1.94856527e-12j
  4.99998253e-01+1.03669667e-08j]]


In [82]:
np.allclose(
    pols2[1][2] * np.exp(1j*np.pi*2* np.dot(hkls2[1], fractional_positions.T))[:, np.newaxis]  * .5/(-4.83545889e-01+1.27217319e-01j),
    pols2[0][2],
)

True

It matches! Note that an overall extra phase factor is fine 

# More testing

In [94]:
hkls2 = [[0,0,0], [1,0,0]]
phonon.set_qpoints_phonon(hkls2, is_eigenvectors=True, write_dynamical_matrices=False)
freqs2, pols2 = phonon.get_qpoints_phonon()
print pols2.shape
pols2 = np.transpose(pols2, (0,2,1))
pols2.shape = 2, 12, 4, 3

(2, 12, 12)


In [95]:
print freqs2.shape
print freqs2[0]
assert np.allclose(freqs2[0], freqs2[1])

(2, 12)
[3.34225831e-02 3.34225831e-02 9.28652901e-02 1.52100901e+00
 1.52100901e+00 2.61040097e+00 2.63887591e+01 2.65558347e+01
 4.76904068e+01 4.76904068e+01 4.78045328e+01 4.78045328e+01]


Energies match apparantly. Next: polarizations

In [96]:
pols2[0][2]

array([[ 3.54763646e-13+0.j, -9.11045243e-13+0.j,  5.00000874e-01+0.j],
       [ 3.56783533e-13+0.j, -9.24304724e-13+0.j,  5.00000874e-01+0.j],
       [ 3.54855214e-13+0.j, -9.10807688e-13+0.j,  4.99999126e-01+0.j],
       [ 3.56935177e-13+0.j, -9.24235284e-13+0.j,  4.99999126e-01+0.j]])

In [97]:
pols2[1][2]

array([[ 2.60927883e-13+0.00000000e+00j, -2.74967258e-12-5.40388288e-12j,
        -4.82792600e-01-1.30046835e-01j],
       [ 2.55083816e-13-1.09040293e-15j, -2.77238309e-12-5.45494847e-12j,
        -4.82792600e-01-1.30046835e-01j],
       [-1.30442351e-13-2.25886655e-13j, -3.30508248e-12+5.08329257e-12j,
         1.28771977e-01+4.83132388e-01j],
       [-1.26573222e-13+2.21437246e-13j,  6.11043195e-12+3.26466818e-13j,
         3.54018918e-01-3.53086012e-01j]])

They don't match. 
Next we adjust the phase factor and see if they match

In [98]:
np.exp(-1j*2*np.pi*np.dot(hkls2[1], fractional_positions.T))

array([ 1. +0.j       ,  1. +0.j       , -0.5-0.8660254j, -0.5+0.8660254j])

In [99]:
print pols2[1][2] * np.exp(1j*np.pi*2* np.dot(hkls2[1], fractional_positions.T))[:, np.newaxis] 

[[ 2.60927883e-13+0.00000000e+00j -2.74967258e-12-5.40388288e-12j
  -4.82792600e-01-1.30046835e-01j]
 [ 2.55083816e-13-1.09040293e-15j -2.77238309e-12-5.45494847e-12j
  -4.82792600e-01-1.30046835e-01j]
 [ 2.60844757e-13-2.30622932e-17j -2.74971926e-12-5.40393167e-12j
  -4.82790910e-01-1.30046390e-01j]
 [ 2.55056892e-13-1.10299717e-15j -2.77248742e-12-5.45502271e-12j
  -4.82790916e-01-1.30046370e-01j]]


In [100]:
np.allclose(
    pols2[1][2] * np.exp(1j*np.pi*2* np.dot(hkls2[1], fractional_positions.T))[:, np.newaxis]  * .5/(-4.82792600e-01-1.30046835e-01j),
    pols2[0][2],
)

True

It matches! Note that an overall extra phase factor is fine 

-j does not work:

In [101]:
print pols2[1][2] * np.exp(-1j*np.pi*2* np.dot(hkls2[1], fractional_positions.T))[:, np.newaxis] 

[[ 2.60927883e-13+0.00000000e+00j -2.74967258e-12-5.40388288e-12j
  -4.82792600e-01-1.30046835e-01j]
 [ 2.55083816e-13-1.09040293e-15j -2.77238309e-12-5.45494847e-12j
  -4.82792600e-01-1.30046835e-01j]
 [-1.30402406e-13+2.25909717e-13j  6.05480174e-12+3.20639106e-13j
   3.54018933e-01-3.53085998e-01j]
 [-1.28483669e-13-2.20334249e-13j -3.33794454e-12+5.12855589e-12j
   1.28771997e-01+4.83132383e-01j]]
